In [4]:
import torch
from diffusers import StableDiffusionPipeline
from safetensors.torch import load_file

# 1. Stable Diffusion 모델 로드
model_id = "runwayml/stable-diffusion-v1-5"  # SD 1.5 모델 사용
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

# 2. LoRA 가중치 로드
lora_path = "/data/ephemeral/home/badge.safetensors"  # 업로드한 LoRA 파일 경로
lora_weights = load_file(lora_path)

# 3. LoRA를 모델에 병합
def apply_lora(pipe, lora_weights, alpha=1.0):
    # 모델의 텍스트 인코더와 U-Net 가중치 업데이트
    for key, value in lora_weights.items():
        layer_name, param_type = key.split(".")[0], key.split(".")[-1]
        
        if "text_encoder" in key:
            layer = pipe.text_encoder
        elif "unet" in key:
            layer = pipe.unet
        else:
            continue

        layer_dict = dict(layer.named_parameters())
        if key in layer_dict:
            layer_dict[key].data += alpha * value.to(layer_dict[key].device)

apply_lora(pipe, lora_weights)

# 4. 이미지 생성
prompt = "octagonal badge with futuristic design"
num_images = 1

images = pipe(prompt, num_images_per_prompt=num_images, guidance_scale=7.5).images

# 5. 결과 저장
for i, img in enumerate(images):
    img.save(f"generated_image_{i}.png")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [6]:
import torch
from PIL import Image
from diffusers import StableDiffusionImg2ImgPipeline
from safetensors.torch import load_file

# 1. Stable Diffusion 모델 로드
model_id = "runwayml/stable-diffusion-v1-5"  # SD 1.5 모델 사용
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

# 2. LoRA 가중치 로드
lora_path = "/data/ephemeral/home/badge.safetensors"  # LoRA 파일 경로
lora_weights = load_file(lora_path)

# 3. LoRA를 모델에 병합
def apply_lora(pipe, lora_weights, alpha=1.0):
    # 모델의 텍스트 인코더와 U-Net 가중치 업데이트
    for key, value in lora_weights.items():
        layer_name, param_type = key.split(".")[0], key.split(".")[-1]
        
        if "text_encoder" in key:
            layer = pipe.text_encoder
        elif "unet" in key:
            layer = pipe.unet
        else:
            continue

        layer_dict = dict(layer.named_parameters())
        if key in layer_dict:
            layer_dict[key].data += alpha * value.to(layer_dict[key].device)

apply_lora(pipe, lora_weights)

# 4. 입력 이미지 로드
input_image_path = "/data/ephemeral/home/test01.jpg"  # 원본 이미지 경로
init_image = Image.open(input_image_path).convert("RGB")
init_image = init_image.resize((512, 512))  # Stable Diffusion 모델의 입력 크기에 맞게 조정

# 5. img2img 생성
prompt = "octagonal badge with futuristic design"  # 텍스트 프롬프트
num_images = 1
strength = 0.8  # 원본 이미지에 대한 영향도 (0.0: 완전히 원본, 1.0: 완전히 새로운 이미지)
guidance_scale = 7.5  # 텍스트 프롬프트에 대한 가중치

images = pipe(prompt=prompt, image=init_image, strength=strength, guidance_scale=guidance_scale, num_images_per_prompt=num_images).images

# 6. 결과 저장
for i, img in enumerate(images):
    img.save(f"generated_img2img_{i}.png")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]